In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
from requests import get

driver = webdriver.Chrome()
#options = webdriver.ChromeOptions()
#options.add_argument('--headless')

#options = webdriver.ChromeOptions() ;
#prefs = {"download.default_directory" : "C:\Tutorial\down"};
#options.add_experimental_option("prefs",prefs);

try:
    
    driver.get('https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx')
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'cboCategorias')))

    
    dropdown = driver.find_element(By.XPATH, '/html/body/form[1]/div[3]/div/fieldset/div[5]/div[1]/div[1]/div/a')
    dropdown.click()

    categoria_dropdown = Select(driver.find_element(By.ID, 'cboCategorias'))
    actions = ActionChains(driver)
    actions.move_to_element(dropdown).click().perform()
    
    time.sleep(3)
    
    search = driver.find_element("xpath",'/html/body/form[1]/div[3]/div/fieldset/div[5]/div[1]/div[1]/div/div/div/input')
    search.send_keys("Fato Relevante")
    search.send_keys(Keys.ENTER)  
      
    time.sleep(3)
    #Seleciona a data para semana
    #data_entrega_dropdown = Select(driver.find_element(By.ID, 'cboEntrega'))
    #data_entrega_dropdown.select_by_visible_text('Na semana')
    #semanal = driver.find_element(By.XPATH, '/html/body/form[1]/div[3]/div/fieldset/div[4]/div[1]/div[1]/div[2]/input')
    #semanal.click()
    
    submit_button = driver.find_element(By.ID, 'btnConsulta')
    submit_button.click()
    time.sleep(15)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'grdDocumentos')))

    #download = driver.find_element(By.CLASS_NAME,"fi-download")
    #download.click()
    #time.sleep(10)
    
    results = driver.find_element(By.ID, 'grdDocumentos')
    s = results.text
    
finally:
    driver.quit()

The chromedriver version (124.0.6367.207) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (125.0.6422.77); currently, chromedriver 125.0.6422.78 is recommended for chrome 125.*, so it is advised to delete the driver in PATH and retry


In [27]:
import re
import datetime

entries = re.split(r'\d{5}-\d\s+', s)[1:]
prefixes = re.findall(r'\d{5}-\d\s+', s)

for prefix, entry in zip(prefixes, entries):
    entry = prefix + entry 

    date_time_match = re.search(r"(\d{2}/\d{2}/\d{4}\s\d{2}:\d{2})", entry)
    if not date_time_match:
        continue

    date_time_str = date_time_match.group(1)
    D, T = date_time_str.split(" ")
    dd, mm, yy = map(int, D.split("/"))
    hr, min = map(int, T.split(":"))

    inst = datetime.datetime(yy, mm, dd).isoweekday()

    if 1 <= inst <= 5 and 10 <= hr <= 17:
        name_match = re.search(r"\d{5}-\d\s+([A-Z\s\.ÇÃÕÁÉÍÓÚÂÊÎÔÛÀÈÌÒÙäëïöüÄËÏÖÜß-]+)\s+Fato Relevante", entry)
        if not name_match:
            continue
        name = name_match.group(1).strip()

        topico_match = re.search(r'(?<=Assunto\(s\):\s)(.*(?:\n\s+.+)*)', entry)
        if not topico_match:
            continue
        topico = topico_match.group(1).strip()

        cleaned_topico = ' '.join(line.strip() for line in topico.split('\n'))

        
    with open("output.txt", "w") as output:
        print("Nome da empresa:", name, file = output)
        print("Tópico:", cleaned_topico, file = output)
        print("No dia:", D, "Ás:", T, "horas.", file = output)
        print("----------------------------", file = output)